# Python で気軽に化学・化学工学
# 第 6 章 データセットの見える化 (可視化) をする
## 6.1 主成分分析 (Principal Component Analysis, PCA)

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

### あやめのデータセット (iris_with_species.csv)
有名な [Fisher’s Iris Data](https://en.wikipedia.org/wiki/Iris_flower_data_set)。150個のあやめについて、がく片長(Sepal Length)、がく片幅(Sepal Width)、花びら長(Petal Length)、花びら幅(Petal Width)が計測されています。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('iris_with_species.csv', index_col=0, header=0) # あやめのデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 数値データの特徴量のみを x に (あやめのデータでは 0 列目が Species でカテゴリーの特徴量であるため、それ以外の特徴量を取り出しています)

特徴量の標準化

In [ ]:
autoscaled_x = (x - x.mean()) / x.std() # 平均を引いてから、標準偏差で割ります。x は DataFrame 型、x.mean(), x.std() は Series 型でデータ型は異なりますが、特徴量の名前が同じであるため、x のすべてのサンプルに対して x.mean() を引き、x.std() で割る計算になります。

## PCA の実行

### scikit-learn
- Python における代表的なデータ解析・機械学習のためのライブラリ
- データセットの可視化、クラスタリング、クラス分類、回帰分析などにおける様々な手法に関するプログラムを利用できる
- 手法を検討するためのサンプルデータを読み込み利用することもできる
- [scikit-learn の公式ウェブサイト](https://scikit-learn.org/stable/)

In [ ]:
from sklearn.decomposition import PCA # scikit-learn の中の PCA を実行するためのライブラリを取り込みます

In [ ]:
pca = PCA() # PCA を行ったり PCA の結果を格納したりするための変数を、pca として宣言します

下はテキスト化してありますが、このように n_components=a とすれば、成分数 a までしか計算しないように設定できます。あやめのデータでは必要ありませんが、特徴量の数が多すぎて、すべての主成分を計算させると計算時間がかかりすぎて終わらない場合は、少ない成分数までに制限するとよいでしょう。

In [ ]:
#pca = PCA(n_components=2)

In [ ]:
pca.fit(autoscaled_x) # 特徴量の標準化後のデータを用いて、PCA を実行

In [ ]:
pca.components_ # ローディングベクトル P。array 型というデータ型で得られます

In [ ]:
pca.components_.T # 本書における式(6.4) における P の行・列と合わせるため、転置します

In [ ]:
loading_vectors = pd.DataFrame(pca.components_.T) # ローディングベクトルを、使い慣れた pandas の DataFrame 型に変換し、loading_vectors という変数とする

In [ ]:
loading_vectors # 念のため確認

In [ ]:
loading_vectors.index = x.columns # P の行の名前を、元の多変量データ x の変数名に

In [ ]:
pc_names = ['PC1', 'PC2', 'PC3', 'PC4'] # 主成分の名前 PC1, PC2, ... 。
# 主成分の数に合わせて作成する必要があります。後の連載で扱う for 文を学べば、もっと効率的に作成可能です

In [ ]:
loading_vectors.columns = pc_names # P の列の名前を、PC1, PC2, ... に

In [ ]:
loading_vectors # 念のため確認

In [ ]:
loading_vectors.to_csv('loading_vectors.csv') # DaraFrame 型であれば、第二回でやったように csv ファイルに保存することもできるわけです

loading_vectors.csv を Excel 等で開いて中身を確認しましょう。

In [ ]:
(loading_vectors ** 2).sum() # 制約条件の通り、(縦に)二乗して足し合わせると 1 になることを確認

下で主成分スコア T の計算をします。今回は主成分分析を実行したデータ autoscaled_x と同じデータに対して主成分スコアを計算していますが、別のデータを入力すれば、そのデータに対する主成分スコアを計算できます。

In [ ]:
pca.transform(autoscaled_x) # 主成分スコア T の計算。array 型で得られます

In [ ]:
score = pd.DataFrame(pca.transform(autoscaled_x)) # データ型を、使い慣れた pandas の DataFrame 型に変換

In [ ]:
score # 念のため確認

In [ ]:
score.index = x.index # スコアのサンプル名を、元のデータセットのサンプル名に

In [ ]:
score.columns = pc_names # スコアの列の名前を、PC1, PC2, ... に

In [ ]:
score # 念のため確認

In [ ]:
score.to_csv('score.csv') # スコアを csv ファイルに保存

score.csv を Excel 等で開いて中身を確認しましょう。

In [ ]:
score.corr() # 主成分の間の相関係数がおよそ 0 になっていることを確認。主成分間に情報の重複がなく、効率的に情報を圧縮できていることがわかります

寄与率、累積寄与率

In [ ]:
pca.explained_variance_ratio_ # 寄与率。array 型で得られます

In [ ]:
contribution_ratios = pd.DataFrame(pca.explained_variance_ratio_) # DataFrame 型に変換

In [ ]:
contribution_ratios.columns = ['contribution_ratio'] # 列名を変更

In [ ]:
contribution_ratios # 念のため確認

In [ ]:
contribution_ratios.to_csv('contribution_ratios.csv') # 寄与率を csv ファイルに保存

In [ ]:
cumulative_contribution_ratios = contribution_ratios.cumsum() # cumsum() で寄与率の累積和を計算

In [ ]:
cumulative_contribution_ratios.columns = ['cumulative_contribution_ratio'] # 列名を変更

In [ ]:
cumulative_contribution_ratios # 累積寄与率

In [ ]:
cumulative_contribution_ratios.to_csv('cumulative_contribution_ratios.csv') # 累積寄与率を csv ファイルに保存

寄与率を棒グラフで、累積寄与率を線で入れたプロット図を重ねて描画します。事前に、x 軸の値を第 2 回に行った range で準備しておきます。

In [ ]:
import matplotlib.pyplot as plt # 描画のためインポート

In [ ]:
x_axis = range(1, contribution_ratios.shape[0] + 1) # 1 から成分数までの整数が x 軸の値

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.bar(x_axis, contribution_ratios.iloc[:, 0], align='center') # 寄与率の棒グラフ
plt.plot(x_axis, cumulative_contribution_ratios.iloc[:, 0], 'r.-') # 累積寄与率の線を入れたプロット図
plt.xlabel('Number of principal components') # 横軸の名前
plt.ylabel('Contribution ratio(blue),\nCumulative contribution ratio(red)') # 縦軸の名前。\n で改行しています
plt.show() # 以上の設定で描画

データセットの可視化

主成分スコアをプロットしてデータセットの確認をします。今回は、component_number_1 番目の主成分と component_number_2 番目の主成分との間の散布図を描画するプログラムとします。

In [ ]:
component_number_1 = 0
component_number_2 = 1

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(score.iloc[:, component_number_1], score.iloc[:, component_number_2]) # 散布図の作成
plt.xlabel(score.columns[component_number_1]) # 横軸の名前。ここでは、component_number_1 番目の列の名前にしています
plt.ylabel(score.columns[component_number_2]) # 縦軸の名前。ここでは、component_number_2 番目の列の名前にしています
plt.show() # 以上の設定において、グラフを描画します

component_number_1, component_number_2 の整数を変えることで、表示する主成分のペアをいろいろと変えてサンプルをプロットしてみましょう。

### 【参考】
下のようにすれば、第 4 章の散布図のときと同様にして、あやめの種類ごとにサンプルの色を変えて描画できます。

In [ ]:
iris_types = dataset.iloc[:, 0] # あやめの種類

In [ ]:
iris_types # 念のため確認

In [ ]:
pd.factorize(iris_types)[0] # あやめの種類を、0, 1, 2 に変換。この数値に基づいて、点の色を変えて散布図を描画します

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(score.iloc[:, component_number_1], score.iloc[:, component_number_2], c=pd.factorize(iris_types)[0], cmap=plt.get_cmap('jet')) # 散布図の作成。あやめの種類ごとにプロットの色を変えています
plt.xlabel(score.columns[component_number_1]) # 横軸の名前。ここでは、component_number_1 番目の列の名前にしています
plt.ylabel(score.columns[component_number_2]) # 縦軸の名前。ここでは、component_number_2 番目の列の名前にしています
plt.show() # 以上の設定において、グラフを描画します

### 【参考】
下のようにすれば、第 4 章の散布図のときと同様にして、すべての主成分間の散布図をいっぺんに描くことができます。対角線のグラフは、各主成分のヒストグラムです。

In [ ]:
plt.rcParams['font.size'] = 10 # 横軸や縦軸の名前の文字などのフォントのサイズ
pd.plotting.scatter_matrix(score)
plt.show()

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。

### 練習問題

データセット `descriptors_8_with_boiling_point.csv` を読み込み、特徴量の標準化をしてから、PCA をしましょう。主成分ごとの寄与率や累積寄与率を確認したり、主成分の散布図を確認したりしてみましょう。一番下にコードの例があります。

### 沸点のデータセット (descriptors_8_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、8 つの特徴量 (記述子) で化学構造が数値化されています。記述子は、分子量 (MolWt)、水素原子以外の原子で計算された分子量 (HeavyAtomMolWt)、価電子の数 (NumValenceElectrons)、水素原子以外の原子の数 (HeavyAtomCount)、窒素原子と酸素原子の数 (NOCount)、水素原子と炭素原子以外の原子の数 (NumHeteroatoms)、回転可能な結合の数 (NumRotatableBonds)、環の数 (RingCount) です。

### 練習問題 コードの例

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0, header=0) # 沸点のデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 分子構造の特徴量のみを x に

特徴量の標準化

In [ ]:
autoscaled_x = (x - x.mean()) / x.std() # 平均を引いてから、標準偏差で割ります。x は DataFrame 型、x.mean(), x.std() は Series 型でデータ型は異なりますが、特徴量の名前が同じであるため、x のすべてのサンプルに対して x.mean() を引き、x.std() で割る計算になります。

PCA

In [ ]:
from sklearn.decomposition import PCA # scikit-learn の中の PCA を実行するためのライブラリを取り込みます

In [ ]:
pca = PCA() # PCA を行ったり PCA の結果を格納したりするための変数を、pca として宣言します

In [ ]:
pca.fit(autoscaled_x) # 特徴量の標準化後のデータを用いて、PCA を実行

In [ ]:
loading_vectors = pd.DataFrame(pca.components_.T) # ローディングベクトルを、使い慣れた pandas の DataFrame 型に変換し、loading_vectors という変数とする

In [ ]:
loading_vectors.index = x.columns # P の行の名前を、元の多変量データ x の変数名に

In [ ]:
loading_vectors # 念のため確認

In [ ]:
loading_vectors.to_csv('loading_vectors_bp.csv') # 保存

In [ ]:
score = pd.DataFrame(pca.transform(autoscaled_x)) # 主成分スコア T を計算し、使い慣れた pandas の DataFrame 型に変換

In [ ]:
score.index = x.index # スコアのサンプル名を、元のデータセットのサンプル名に

In [ ]:
score # 念のため確認

In [ ]:
score.to_csv('score_bp.csv') # スコアを csv ファイルに保存

寄与率、累積寄与率

In [ ]:
contribution_ratios = pd.DataFrame(pca.explained_variance_ratio_) # DataFrame 型に変換

In [ ]:
contribution_ratios.columns = ['contribution_ratio'] # 列名を変更

In [ ]:
contribution_ratios # 念のため確認

In [ ]:
contribution_ratios.to_csv('contribution_ratios_bp.csv') # 寄与率を csv ファイルに保存

In [ ]:
cumulative_contribution_ratios = contribution_ratios.cumsum() # cumsum() で寄与率の累積和を計算

In [ ]:
cumulative_contribution_ratios.columns = ['cumulative_contribution_ratio'] # 列名を変更

In [ ]:
cumulative_contribution_ratios # 念のため確認

In [ ]:
cumulative_contribution_ratios.to_csv('cumulative_contribution_ratios.csv') # 累積寄与率を csv ファイルに保存

寄与率を棒グラフで、累積寄与率を線で入れたプロット図を重ねて描画

In [ ]:
import matplotlib.pyplot as plt # 描画のためインポート

In [ ]:
x_axis = range(1, contribution_ratios.shape[0] + 1) # 1 から成分数までの整数が x 軸の値

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.bar(x_axis, contribution_ratios.iloc[:, 0], align='center') # 寄与率の棒グラフ
plt.plot(x_axis, cumulative_contribution_ratios.iloc[:, 0], 'r.-') # 累積寄与率の線を入れたプロット図
plt.xlabel('Number of principal components') # 横軸の名前
plt.ylabel('Contribution ratio(blue),\nCumulative contribution ratio(red)') # 縦軸の名前。\n で改行しています
plt.show() # 以上の設定で描画

データセットの可視化

主成分スコアをプロットしてデータセットの確認をします。今回は、component_number_1 番目の主成分と component_number_2 番目の主成分との間の散布図を描画するプログラムとします。

In [ ]:
component_number_1 = 0
component_number_2 = 1

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(score.iloc[:, component_number_1], score.iloc[:, component_number_2]) # 散布図の作成
plt.xlabel(score.columns[component_number_1]) # 横軸の名前。ここでは、component_number_1 番目の列の名前にしています
plt.ylabel(score.columns[component_number_2]) # 縦軸の名前。ここでは、component_number_2 番目の列の名前にしています
plt.show() # 以上の設定において、グラフを描画します

component_number_1, component_number_2 の整数を変えることで、表示する主成分のペアをいろいろと変えてサンプルをプロットしてみましょう。

In [ ]:
boiling_point = dataset.iloc[:, 0] # 沸点

In [ ]:
boiling_point # 念のため確認

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(score.iloc[:, component_number_1], score.iloc[:, component_number_2], c=boiling_point, cmap=plt.get_cmap('jet')) # 散布図の作成。あやめの種類ごとにプロットの色を変えています
plt.xlabel(score.columns[component_number_1]) # 横軸の名前。ここでは、component_number_1 番目の列の名前にしています
plt.ylabel(score.columns[component_number_2]) # 縦軸の名前。ここでは、component_number_2 番目の列の名前にしています
plt.colorbar() # カラーバーを表示します
plt.show() # 以上の設定において、グラフを描画します

沸点の値の近い化合物が、プロット上でも近くに分布している傾向があることが確認できます